In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda, float64, complex128
from numba.cuda import jit as cuda_jit
import math

import few

from few.trajectory.inspiral import EMRIInspiral
from few.trajectory.ode import KerrEccEqFlux
from few.amplitude.ampinterp2d import AmpInterpKerrEccEq
from few.summation.interpolatedmodesum import InterpolatedModeSum 


from few.utils.ylm import GetYlms

from few import get_file_manager

from few.waveform import FastKerrEccentricEquatorialFlux, GenerateEMRIWaveform

from few.utils.geodesic import get_fundamental_frequencies

from few.utils.constants import YRSID_SI

import os
import sys

# Change to the desired directory
os.chdir('/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

# Add it to Python path
sys.path.insert(0, '/nfs/home/svu/e1498138/localgit/FEWNEW/work/')

import GWfuncs
import loglike
import modeselector
import dynesty
# import gc
# import pickle
import cupy as cp

# tune few configuration
cfg_set = few.get_config_setter(reset=True)
cfg_set.set_log_level("info")



# Initialization

In [2]:
for backend in ["cpu", "cuda11x", "cuda12x", "cuda", "gpu"]: 
    print(f" - Backend '{backend}': {"available" if few.has_backend(backend) else "unavailable"}")  

 - Backend 'cpu': available
 - Backend 'cuda11x': unavailable
 - Backend 'cuda12x': available
 - Backend 'cuda': available
 - Backend 'gpu': available


In [ ]:
# GPU configuration 
use_gpu = True
force_backend = "cuda12x"  
dt = 10     # Time step
T = 0.25     # Total time


In [ ]:
# keyword arguments for inspiral generator 
inspiral_kwargs={
        "func": 'KerrEccEqFlux',
        "DENSE_STEPPING": 0, #change to 1/True for uniform sampling
        "include_minus_m": True, 
}

# keyword arguments for inspiral generator 
amplitude_kwargs = {
    "force_backend": force_backend,
    # "use_gpu" : use_gpu
}

# keyword arguments for Ylm generator (GetYlms)
Ylm_kwargs = {
    "force_backend": force_backend,
    # "assume_positive_m": True  # if we assume positive m, it will generate negative m for all m>0
}

# keyword arguments for summation generator (InterpolatedModeSum)
sum_kwargs = {
    "force_backend":force_backend,
    "pad_output": True,
    # "separate_modes": True
    # "use_gpu" : use_gpu
}

print("Creating GenerateEMRIWaveform class...")
# # Kerr eccentric flux
# waveform_gen = FastKerrEccentricEquatorialFlux(
#     inspiral_kwargs=inspiral_kwargs,
#     amplitude_kwargs=amplitude_kwargs,
#     Ylm_kwargs=Ylm_kwargs,
#     sum_kwargs=sum_kwargs,
#     use_gpu=use_gpu,
# )

waveform_gen = GenerateEMRIWaveform(
    FastKerrEccentricEquatorialFlux, 
    frame='detector',
    inspiral_kwargs=inspiral_kwargs, 
    amplitude_kwargs=amplitude_kwargs, 
    Ylm_kwargs=Ylm_kwargs,
    sum_kwargs=sum_kwargs,
    use_gpu=use_gpu
)

Setting up waveform generator...
Creating GenerateEMRIWaveform class...


In [5]:
waveform_gen.waveform_generator.inspiral_generator

In [6]:
gwf = GWfuncs.GravWaveAnalysis(T, dt)

In [7]:
#Generating data (true)

m1 = 1e6
m2 = 3e1
a = 0.7
p0 = 7.5
e0 = 0.4
xI0 = 1.0 #NOTE: fixed, equatorial
dist = 3 
qS = 0.5 
phiS = 1
qK = 1 #NOTE: fixed, degenerate
phiK = 1.5 #NOTE: fixed, degenerate
# Phases
Phi_phi0 = 0.4
Phi_theta0 = 0.0 # NOTE: fixed, equatorial
Phi_r0 = 0.5

params_star = (m1, m2, a, p0, e0, xI0, dist, qS, phiS, qK, phiK, Phi_phi0, Phi_theta0, Phi_r0)

In [38]:
loglike_obj = loglike.LogLike(params_star, waveform_gen, gwf, M_init=30, verbose=True, distance_threshold=1.5)

In [39]:
loglike_obj.signal

array([ 1.95933653e-22+1.40786933e-22j,  1.55210660e-22+1.83191352e-22j,
        1.07150926e-22+2.12935249e-22j, ...,
       -4.83254991e-23+2.39727360e-22j, -1.08697091e-22+2.06112223e-22j,
       -1.53979464e-22+1.61038617e-22j])

In [10]:
# # Generate sample template
# m1_t = 1.5e6
# m2_t = 1.5e1
# a_t = 0.5
# p0_t = 13
# e0_t = 0.2
# xI_t = 1.0
# theta_t = np.pi/3  # polar viewing angle
# phi_t = np.pi/4  # azimuthal viewing angle
# dist_t = 1 # Gpc
# params_template = (m1_t, m2_t, a_t, p0_t, e0_t, xI_t, theta_t, phi_t, dist_t)

In [11]:
# params_star = (m1, m2, a, p0, e0, xI0, dist, qS, phiS, qK, phiK, Phi_phi0, Phi_theta0, Phi_r0)

# Example search for debugging

In [12]:
def parameter_space_search_example(n_samples=10):
    print(f"Running parameter space search with {n_samples} samples...")

    # 3sigma
    # intrinsic parameters ranges
    m1_range = (9.9999999987e+05, 1.0000000001e+06)
    m2_range = (2.9999921753e+01, 3.0000078247e+01)
    a_range = (6.9989929349e-01, 7.0010070651e-01)
    p0_range = (7.4994459434e+00, 7.5005540566e+00)
    e0_range = (3.9999126130e-01, 4.0000873870e-01) #1sigma
    dist_range = (2.9566301838e+00, 3.0433698162e+00)
    # detector frame angles ranges
    qS_range = (4.7685409100e-01, 5.2314590900e-01)   
    phiS_range = (9.7607316244e-01, 1.0239268376e+00) 
    # phase ranges
    Phi_phi0_range = (3.6664072881e-01, 4.3335927119e-0)    
    Phi_r0_range = (4.8936765646e-01, 5.1063234354e-01)     

    # Seed for reproducibility
    np.random.seed(7)

    for i in range(n_samples):
        # Sample masses log-uniformly, others uniformly
        m1 = 10**(np.random.uniform(np.log10(m1_range[0]),np.log10(m1_range[1])))
        m2 = 10**(np.random.uniform(np.log10(m2_range[0]),np.log10(m2_range[1])))
        params = np.array([
            m1,
            m2,
            np.random.uniform(*a_range),
            np.random.uniform(*p0_range),
            np.random.uniform(*e0_range),
            xI0,  # xI0 fixed, equatorial
            np.random.uniform(*dist_range),
            np.random.uniform(*qS_range),
            np.random.uniform(*phiS_range),
            qK, # qK fixed, degenerate
            phiK, # phiK fixed, degenerate
            np.random.uniform(*Phi_phi0_range),
            Phi_theta0,  # Phi_theta0 fixed, equatorial 
            np.random.uniform(*Phi_r0_range)
        ])

        try:
            # Evaluate likelihood
            print(f" === Parameters for sample {i+1}: {params} ===")
            f_stat = loglike_obj(params)
            print(f"Sample {i+1}/{n_samples}: f_stat = {f_stat}")

        except Exception as e:
            print(f"Error in evaluation {i+1}: {e}")
            continue

In [13]:
%%time
parameter_space_search_example(n_samples=1)

Running parameter space search with 1 samples...
 === Parameters for sample 1: [1.00000000e+06 3.00000438e+01 6.99987595e-01 7.50024762e+00
 4.00008354e-01 1.00000000e+00 3.00333912e+00 5.00051868e-01
 9.79521074e-01 1.00000000e+00 1.50000000e+00 1.43152527e+00
 0.00000000e+00 4.99997501e-01] ===
Generating new modes...
Initial mode selected: (2, -2, 1) with power 276.0691045897627
Considering mode 1 / 30 : (2, -2, -1) with power 167.3257332470361
 - Overlap with selected mode 0: 0.0002303662947770552
Considering mode 2 / 30 : (2, -2, 0) with power 162.9556849835556
 - Overlap with selected mode 0: 0.00016921667653693406
 - Overlap with selected mode 1: 0.00015151693375683943
Considering mode 3 / 30 : (2, -2, 2) with power 135.8345605891191
 - Overlap with selected mode 0: 0.0002556340541046812
 - Overlap with selected mode 1: 0.0
 - Overlap with selected mode 2: 0.00016673913166406263
Considering mode 4 / 30 : (2, -2, 3) with power 43.65790092649449
 - Overlap with selected mode 0: 2.

In [14]:
%%time
parameter_space_search_example(n_samples=10)

Running parameter space search with 10 samples...
 === Parameters for sample 1: [1.00000000e+06 3.00000438e+01 6.99987595e-01 7.50024762e+00
 4.00008354e-01 1.00000000e+00 3.00333912e+00 5.00051868e-01
 9.79521074e-01 1.00000000e+00 1.50000000e+00 1.43152527e+00
 0.00000000e+00 4.99997501e-01] ===
Parameter distance: 0.0
Using cached modes...
Xdotrho: (13.965107923274793+0j)
Sample 1/10: f_stat = 0.2579222885539244
 === Parameters for sample 2: [1.00000000e+06 3.00000475e+01 6.99976020e-01 7.49951901e+00
 3.99996297e-01 1.00000000e+00 3.03552799e+00 4.86732087e-01
 9.97708956e-01 1.00000000e+00 1.50000000e+00 4.06069030e+00
 0.00000000e+00 4.89897131e-01] ===
Parameter distance: 2.62947820301638
Generating new modes...
Initial mode selected: (2, -2, 1) with power 275.19049017298
Considering mode 1 / 30 : (2, -2, -1) with power 166.79474446776518
 - Overlap with selected mode 0: 0.00023039490034481162
Considering mode 2 / 30 : (2, -2, 0) with power 162.48950703826554
 - Overlap with sel

In [15]:
%%time
parameter_space_search_example(n_samples=100)

Running parameter space search with 100 samples...
 === Parameters for sample 1: [1.00000000e+06 3.00000438e+01 6.99987595e-01 7.50024762e+00
 4.00008354e-01 1.00000000e+00 3.00333912e+00 5.00051868e-01
 9.79521074e-01 1.00000000e+00 1.50000000e+00 1.43152527e+00
 0.00000000e+00 4.99997501e-01] ===
Parameter distance: 0.4823835378211443
Using cached modes...
Xdotrho: (13.965107923274793+0j)
Sample 1/100: f_stat = 0.2579222885539244
 === Parameters for sample 2: [1.00000000e+06 3.00000475e+01 6.99976020e-01 7.49951901e+00
 3.99996297e-01 1.00000000e+00 3.03552799e+00 4.86732087e-01
 9.97708956e-01 1.00000000e+00 1.50000000e+00 4.06069030e+00
 0.00000000e+00 4.89897131e-01] ===
Parameter distance: 3.110722270512747
Generating new modes...
Initial mode selected: (2, -2, 1) with power 275.19049017298
Considering mode 1 / 30 : (2, -2, -1) with power 166.79474446776518
 - Overlap with selected mode 0: 0.00023039490034481162
Considering mode 2 / 30 : (2, -2, 0) with power 162.48950703826554
 

# Actual nested sampling

In [41]:
def prior_transform(utheta):
      um1, um2, ua, up0, ue0, udist, uqS, uphiS, uPhi_phi0, uPhi_r0 = utheta

      # 3sigma ranges (exc e0)
      m1_range = (9.9999999987e+05, 1.0000000001e+06)
      m2_range = (2.9999921753e+01, 3.0000078247e+01)
      a_range = (6.9989929349e-01, 7.0010070651e-01)
      p0_range = (7.4994459434e+00, 7.5005540566e+00)
      e0_range = (3.9999126130e-01, 4.0000873870e-01)
      dist_range = (2.9566301838e+00, 3.0433698162e+00)
      qS_range = (4.7685409100e-01, 5.2314590900e-01)
      phiS_range = (9.7607316244e-01, 1.0239268376e+00)
      Phi_phi0_range = (3.6664072881e-01, 4.3335927119e-01)
      Phi_r0_range = (4.8936765646e-01, 5.1063234354e-01)

      # Log-uniform for masses
      m1 = 10**(np.log10(m1_range[0]) + um1 * (np.log10(m1_range[1]) - np.log10(m1_range[0])))
      m2 = 10**(np.log10(m2_range[0]) + um2 * (np.log10(m2_range[1]) - np.log10(m2_range[0])))

      # Uniform for other parameters
      a = (a_range[1] - a_range[0]) * ua + a_range[0]
      p0 = (p0_range[1] - p0_range[0]) * up0 + p0_range[0]
      e0 = (e0_range[1] - e0_range[0]) * ue0 + e0_range[0]
      xI0 = 1.0  # Fixed value
      dist = (dist_range[1] - dist_range[0]) * udist + dist_range[0]
      qS = (qS_range[1] - qS_range[0]) * uqS + qS_range[0]
      phiS = (phiS_range[1] - phiS_range[0]) * uphiS + phiS_range[0]
      qK = 1.0  # Fixed value
      phiK = 1.5  # Fixed value
      Phi_phi0 = (Phi_phi0_range[1] - Phi_phi0_range[0]) * uPhi_phi0 + Phi_phi0_range[0]
      Phi_theta0 = 0.0  # Fixed value
      Phi_r0 = (Phi_r0_range[1] - Phi_r0_range[0]) * uPhi_r0 + Phi_r0_range[0]

      return m1, m2, a, p0, e0, xI0, dist, qS, phiS, qK, phiK, Phi_phi0, Phi_theta0, Phi_r0

In [44]:
rstate = np.random.default_rng(7)
with dynesty.pool.Pool(16, loglike_obj, prior_transform) as pool:
    dsampler = dynesty.DynamicNestedSampler(
        loglike_obj,  
        prior_transform,
        ndim=10,
        bound='multi',
        sample='rwalk',
        rstate=rstate
    )
    dsampler.run_nested()

0it [00:00, ?it/s]

Generating new modes...


Traceback (most recent call last):
  File "/scratch/e1498138/anaconda3/envs/fewsm/lib/python3.12/site-packages/dynesty/dynesty.py", line 913, in __call__
    return self.func(np.asarray(x).copy(), *self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/nfs/home/svu/e1498138/localgit/FEWNEW/work/loglike.py", line 138, in __call__
    selected_modes, selected_labels = mode_selector.select_modes(
                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/nfs/home/svu/e1498138/localgit/FEWNEW/work/modeselector.py", line 267, in select_modes
    total_power = self.gwf.calc_power(self.teuk_modes, self.ylms, m0mask)
                                      ^^^^^^^^^^^^^^^
  File "/nfs/home/svu/e1498138/localgit/FEWNEW/work/modeselector.py", line 86, in teuk_modes
    self._teuk_modes = self.amp(a, p, e, x)
                       ^^^^^^^^^^^^^^^^^^^^
  File "/nfs/home/svu/e1498138/localgit/FEW_SM/FastEMRIWaveforms/src/few/amplitude

Exception while calling loglikelihood function:
  params: [1.00000000e+06 3.00000622e+01 7.00055527e-01 7.49969550e+00
 3.99996507e-01 1.00000000e+00 3.03240189e+00 4.77097832e-01
 1.01537196e+00 1.00000000e+00 1.50000000e+00 4.19820039e-01
 0.00000000e+00 4.99318147e-01]
  args: []
  kwargs: {}
  exception:


OutOfMemoryError: Out of memory allocating 559,440,384 bytes (allocated so far: 12,227,772,416 bytes).

# Visualization

In [ ]:
import os
import pickle
import dynesty
os.chdir('/home/svu/e1498138/localgit/FEWNEW/work/')

In [ ]:
with open('nestedsampling_results.pkl', 'rb') as f:
    dres = pickle.load(f) # deserialize using load()

In [ ]:
dres.summary()

In [ ]:
from dynesty import utils as dyfunc

samples, weights = dres.samples, dres.importance_weights()
mean, cov = dyfunc.mean_and_cov(samples, weights)
ndim = len(mean)
imp_weights = dres.importance_weights()

In [ ]:
#cornerplot
from dynesty import plotting as dyplot
import matplotlib.pyplot as plt
fig, axes = dyplot.cornerplot(dres, show_titles=True, truth_color='black', smooth=0.03, color='midnightblue', quantiles_2d = [0.4, 0.85],
                              title_kwargs={'y': 1.04,'fontsize':25}, title_fmt='.2e',
                              fig=plt.subplots(ndim, ndim, figsize=(30, 30)))
